In [1]:
import sys
sys.path.insert(0, "..")

In [2]:
from source import wino_loader

wino_loader.extract_captions()

In [1]:
import torch 
import pickle 
import clip
import os
from PIL import Image

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [05:06<00:00, 1.16MiB/s]


In [18]:
img = Image.open('../dataset/images/ex_0_img_0.png')
text = ['an old person kisses a young person,a young person kisses an old person']

In [19]:
from transformers import CLIPProcessor, CLIPModel
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
input = processor(text=text, images=img, padding=True )

In [16]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
outputs = model(**input)
logits_per_image = outputs.logits_per_image
probs = logits_per_image.softmax(dim=1)

In [21]:
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)
outputs = model(**inputs)
logits_per_image = outputs.logits_per_image
probs = logits_per_image.softmax(dim=1)

In [22]:
logits_per_image

tensor([[24.5701, 19.3049]], grad_fn=<TBackward0>)

In [23]:
probs

tensor([[0.9949, 0.0051]], grad_fn=<SoftmaxBackward0>)

In [30]:
from PIL import Image
import requests
import torch 
from transformers import CLIPProcessor, CLIPModel
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = [Image.open('../dataset/images/ex_0_img_0.png'), Image.open('../dataset/images/ex_0_img_1.png')]
inputs = processor(text=["an old person kisses a young person","a young person kisses an old person"], images=image, return_tensors="pt", padding=True)
with torch.inference_mode():
    outputs = model(**inputs)

logits_per_image = outputs.logits_per_image
logits_per_text = outputs.logits_per_text
probs = logits_per_image.softmax(dim=1)
probs

tensor([[0.3177, 0.6823],
        [0.2370, 0.7630]])

LOAD EDITED CAPTION FROM EXCEL SHEET

In [1]:

import pandas as pd
import xlrd
dir = "../dataset/edited_captions.xls"
# Here 0th column will be extracted
df = pd.read_excel(dir,
                   index_col = 0)

In [2]:
df

,image_0,image_1,caption_0,caption_1,modified_caption_0,modified_caption_1
img_id,,,,,,
0,ex_0_img_0,ex_0_img_1,an old person kisses a young person,a young person kisses an old person,an old guy kisses a child,a child kisses an old lady
1,ex_1_img_0,ex_1_img_1,the taller person hugs the shorter person,the shorter person hugs the taller person,the taller man hugs the shorter woman,the shorter woman hugs the taller man
2,ex_2_img_0,ex_2_img_1,the masked wrestler hits the unmasked wrestler,the unmasked wrestler hits the masked wrestler,the masked wrestler hits his opponent,the unmasked wrestler hits his opponent
3,ex_3_img_0,ex_3_img_1,a person watches an animal,an animal watches a person,a person watches a whale,a lion watches a person
4,ex_4_img_0,ex_4_img_1,the person without earrings pays the person wi...,the person with earrings pays the person witho...,the guy pays the lady with earrings,the lady with earrings pays another person
...,...,...,...,...,...,...
395,ex_395_img_0,ex_395_img_1,They have sunglasses over their hair,They have hair over their sunglasses,NaN,NaN
396,ex_396_img_0,ex_396_img_1,an odd number of sweet things on a plate with ...,an even number of sweet things on a plate with...,NaN,NaN
397,ex_397_img_0,ex_397_img_1,a large living thing in front of a large non-l...,a large non-living thing in front of a large l...,NaN,NaN


In [18]:
for i in range(51):
    cap_0=df['modified_caption_1'][50]
    break
cap_0

'the girl with red pant is running quite slowly and the guy with green pant runs runs faster'